In [2]:
:dep darn
:dep ndarray
:dep ndarray-csv
:dep polars = { version = "x", features = ["csv","describe","fmt","lazy","ndarray","dtype-categorical","to_dummies"]}
:dep plotters = {version = "0.3.5", features = ["evcxr", "line_series"]}
:dep linfa
:dep linfa-preprocessing
:sccache 1
    
extern crate csv;

use std::io::prelude::*;
use std::fs::*;
use std::path::Path
use ndarray::prelude::*;
use ndarray_csv::Array2Reader;
use ndarray::*;

use polars::prelude::*;
use std::fs::File;

use linfa::*;
use linfa::prelude::*;

use linfa::traits::Transformer;
use linfa_preprocessing::norm_scaling::NormScaler;
use linfa_preprocessing::linear_scaling::LinearScaler;
    
extern crate plotters;
use plotters::prelude::*;
use plotters::series::*;


sccache: true. Warning: dynamic linking disabled, use :cache instead to preserve dynamic linking


In [13]:
//Load CSV datafram
let mut database_rd = csv::Reader::from_path(Path::new("./db/credit_data.csv"))?; 
//Convert csv to multidimension array
let data: Array2<String> = database_rd.deserialize_array2_dynamic()?;
//show frame on html table format
darn::show_frame(&data,None);



Error: mismatched types

In [15]:
//load from polars
let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

//print dataframe
println!("{:?}",data_frame_polars);
//header data
println!("{}",data_frame_polars.head(Some(2)));
//tail data
println!("{}",data_frame_polars.tail(Some(5)));
//filter data
let filter = data_frame_polars.clone().lazy().filter(col("income").gt_eq(1.377630)).collect();
println!("{:?}",filter);





shape: (2_000, 5)
┌──────────┬──────────────┬───────────┬─────────────┬─────────┐
│ clientid ┆ income       ┆ age       ┆ loan        ┆ default │
│ ---      ┆ ---          ┆ ---       ┆ ---         ┆ ---     │
│ i64      ┆ f64          ┆ f64       ┆ f64         ┆ i64     │
╞══════════╪══════════════╪═══════════╪═════════════╪═════════╡
│ 1        ┆ 66155.925095 ┆ 59.017015 ┆ 8106.532131 ┆ 0       │
│ 2        ┆ 34415.153966 ┆ 48.117153 ┆ 6564.745018 ┆ 0       │
│ 3        ┆ 57317.170063 ┆ 63.108049 ┆ 8020.953296 ┆ 0       │
│ 4        ┆ 42709.534201 ┆ 45.751972 ┆ 6103.64226  ┆ 0       │
│ 5        ┆ 66952.688845 ┆ 18.584336 ┆ 8770.099235 ┆ 1       │
│ …        ┆ …            ┆ …         ┆ …           ┆ …       │
│ 1996     ┆ 59221.044874 ┆ 48.518179 ┆ 1926.729397 ┆ 0       │
│ 1997     ┆ 69516.127573 ┆ 23.162104 ┆ 3503.176156 ┆ 0       │
│ 1998     ┆ 44311.449262 ┆ 28.017167 ┆ 5522.786693 ┆ 1       │
│ 1999     ┆ 43756.056605 ┆ 63.971796 ┆ 1622.722598 ┆ 0       │
│ 2000     ┆ 69436.579

In [161]:
//unique data
let df= data_frame_polars.clone().unique(Some(&["default".to_string()]),UniqueKeepStrategy::First, None);
println!("{:?}",df);

let df= data_frame_polars.clone().group_by(["default"])?.select(["default"]).count();
println!("{:?}",df);

let ndarray = df.unwrap().to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap();
// let tuples: Vec<(f64, f64)> = ndarray
//     .rows()
//     .into_iter()
//     .map(|row| (row[0], row[1]))
//     .collect();


evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count Test", ("sans-serif", 50.0))
        .build_cartesian_2d(0u32..2u32, 0f64..2000f64)?;  
    
 chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .y_labels(5)
        .x_label_formatter(&|x| format!("{}", *x))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(4)
            .data(ndarray
                .rows()
                .into_iter()
                .map(|row|((row[0]) as u32, row[1]))))?;

    
    Ok(())
}).style("width:100%")

Ok(shape: (2, 5)
┌──────────┬──────────────┬───────────┬─────────────┬─────────┐
│ clientid ┆ income       ┆ age       ┆ loan        ┆ default │
│ ---      ┆ ---          ┆ ---       ┆ ---         ┆ ---     │
│ i64      ┆ f64          ┆ f64       ┆ f64         ┆ i64     │
╞══════════╪══════════════╪═══════════╪═════════════╪═════════╡
│ 1        ┆ 66155.925095 ┆ 59.017015 ┆ 8106.532131 ┆ 0       │
│ 5        ┆ 66952.688845 ┆ 18.584336 ┆ 8770.099235 ┆ 1       │
└──────────┴──────────────┴───────────┴─────────────┴─────────┘)
Ok(shape: (2, 2)
┌─────────┬───────────────┐
│ default ┆ default_count │
│ ---     ┆ ---           │
│ i64     ┆ u32           │
╞═════════╪═══════════════╡
│ 0       ┆ 1717          │
│ 1       ┆ 283           │
└─────────┴───────────────┘)


Count Test
 
 
 
0
 
 
 
500
 
 
 
1000
 
 
 
1500
 
 
 
2000
 
 
 
 
0
 
 
 
1
 
 
 
2

In [169]:

let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();


let df= data_frame_polars.clone().lazy().select([col("age").cast(DataType::Int32).alias("cast_age")]).with_columns([col("cast_age").fill_null(lit(0))]).collect();
println!("{:?}",df);
let group= df?.group_by(["cast_age"])?.select(["cast_age"]).count()?;
println!("{:?}",group);

let min= group.clone().column("cast_age")?.min::<i32>().unwrap().unwrap();
let max= group.clone().column("cast_age")?.max::<i32>().unwrap().unwrap();

let minY= group.clone().column("cast_age_count")?.min::<i32>().unwrap().unwrap();
let maxY= group.clone().column("cast_age_count")?.max::<i32>().unwrap().unwrap();

let ndarray = group.clone().to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();

evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count age", ("sans-serif", 10.0))
        .build_cartesian_2d(min..max, minY..maxY)?;  
    
 chart.configure_mesh()
        .x_label_formatter(&|x| format!("{}", *x))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(0)
            .data(ndarray
                .rows()
                .into_iter()
                .map(|row|(row[0], row[1]  ))))?;

    
    Ok(())
}).style("width:100%")

Ok(shape: (2_000, 1)
┌──────────┐
│ cast_age │
│ ---      │
│ i32      │
╞══════════╡
│ 59       │
│ 48       │
│ 63       │
│ 45       │
│ 18       │
│ …        │
│ 48       │
│ 23       │
│ 28       │
│ 63       │
│ 56       │
└──────────┘)
shape: (50, 2)
┌──────────┬────────────────┐
│ cast_age ┆ cast_age_count │
│ ---      ┆ ---            │
│ i32      ┆ u32            │
╞══════════╪════════════════╡
│ 27       ┆ 54             │
│ 21       ┆ 44             │
│ 24       ┆ 40             │
│ 0        ┆ 3              │
│ 30       ┆ 33             │
│ …        ┆ …              │
│ 59       ┆ 38             │
│ 62       ┆ 45             │
│ 38       ┆ 34             │
│ 47       ┆ 53             │
│ -36      ┆ 1              │
└──────────┴────────────────┘


Count age
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
10
 
 
 
20
 
 
 
30
 
 
 
40
 
 
 
50
 
 
 
60
 
 
 
 
-40
 
 
 
-20
 
 
 
0
 
 
 
20
 
 
 
40
 
 
 
60

In [144]:

let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();


let df= data_frame_polars.clone().lazy()
    .select([col("income").cast(DataType::Int32).alias("cast_income") / lit(1000)])
    .with_columns([col("cast_income").fill_null(lit(0))]).collect();

println!("{:?}",df);
let group= df?.group_by(["cast_income"])?.select(["cast_income"]).count()?;
println!("{:?}",group);

let min= group.clone().column("cast_income")?.min::<i32>().unwrap().unwrap();
let max= group.clone().column("cast_income")?.max::<i32>().unwrap().unwrap();

let minY= group.clone().column("cast_income_count")?.min::<i32>().unwrap().unwrap();
let maxY= group.clone().column("cast_income_count")?.max::<i32>().unwrap().unwrap();

let ndarray = group.clone().to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();

evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count icome", ("sans-serif", 10.0))
        .build_cartesian_2d(min..max, minY..maxY)?;  
    
 chart.configure_mesh()
        .x_label_formatter(&|x| format!("{}", *x * 1000))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;

    let result = ndarray
                .rows()
                .into_iter()
                .map(|row|(row[0], row[1] ));

 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(0)
            .data(result))?;

    
    Ok(())
}).style("width:100%")

Ok(shape: (2_000, 1)
┌─────────────┐
│ cast_income │
│ ---         │
│ i32         │
╞═════════════╡
│ 66          │
│ 34          │
│ 57          │
│ 42          │
│ 66          │
│ …           │
│ 59          │
│ 69          │
│ 44          │
│ 43          │
│ 69          │
└─────────────┘)
shape: (50, 2)
┌─────────────┬───────────────────┐
│ cast_income ┆ cast_income_count │
│ ---         ┆ ---               │
│ i32         ┆ u32               │
╞═════════════╪═══════════════════╡
│ 21          ┆ 34                │
│ 27          ┆ 41                │
│ 24          ┆ 39                │
│ 30          ┆ 44                │
│ 57          ┆ 43                │
│ …           ┆ …                 │
│ 65          ┆ 38                │
│ 59          ┆ 26                │
│ 53          ┆ 40                │
│ 41          ┆ 42                │
│ 68          ┆ 39                │
└─────────────┴───────────────────┘


Count icome
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
30
 
 
 
35
 
 
 
40
 
 
 
45
 
 
 
50
 
 
 
55
 
 
 
 
20000
 
 
 
25000
 
 
 
30000
 
 
 
35000
 
 
 
40000
 
 
 
45000
 
 
 
50000
 
 
 
55000
 
 
 
60000
 
 
 
65000

In [122]:
let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();


let df= data_frame_polars.clone().lazy()
    .select([col("income").cast(DataType::Int32).alias("cast_income"),
             col("age").cast(DataType::Int32).alias("cast_age"),col("default")])
    .with_columns([col("cast_income").fill_null(lit(0)),col("cast_age").fill_null(lit(0)),col("default").fill_null(lit(0))]).collect().unwrap();

println!("{:?}",df);
// let group= df?.group_by(["cast_income","cast_age"])?.select(["cast_income","cast_age"]).count()?;
// println!("{:?}",group);

let minX= df.clone().column("cast_age")?.min::<i32>().unwrap().unwrap();
let maxX= df.clone().column("cast_age")?.max::<i32>().unwrap().unwrap();

let minY= df.clone().column("cast_income")?.min::<i32>().unwrap().unwrap();
let maxY= df.clone().column("cast_income")?.max::<i32>().unwrap().unwrap();

let ndarray = df.clone().to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();
evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Scatter agexloanxdefault", ("sans-serif", 10.0))
        .build_cartesian_2d(minX..maxX, minY..maxY)?;  
    
 chart.configure_mesh()
        .x_label_formatter(&|x| format!("{}", *x))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


 chart.draw_series(
    ndarray
                .rows()
                .into_iter()
                .map(|point| {
                        
                    let mut color = &BLUE;
                    if(point[2] == 1){
                        color = &RED;
                    }
                    
                    Circle::new((point[1],point[0]), 5, color)
                }
     ));

    
    Ok(())
}).style("width:100%")


shape: (2_000, 3)
┌─────────────┬──────────┬─────────┐
│ cast_income ┆ cast_age ┆ default │
│ ---         ┆ ---      ┆ ---     │
│ i32         ┆ i32      ┆ i64     │
╞═════════════╪══════════╪═════════╡
│ 66155       ┆ 59       ┆ 0       │
│ 34415       ┆ 48       ┆ 0       │
│ 57317       ┆ 63       ┆ 0       │
│ 42709       ┆ 45       ┆ 0       │
│ 66952       ┆ 18       ┆ 1       │
│ …           ┆ …        ┆ …       │
│ 59221       ┆ 48       ┆ 0       │
│ 69516       ┆ 23       ┆ 0       │
│ 44311       ┆ 28       ┆ 1       │
│ 43756       ┆ 63       ┆ 0       │
│ 69436       ┆ 56       ┆ 0       │
└─────────────┴──────────┴─────────┘


Scatter agexloanxdefault
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
25000
 
 
 
30000
 
 
 
35000
 
 
 
40000
 
 
 
45000
 
 
 
50000
 
 
 
55000
 
 
 
60000
 
 
 
65000
 
 
 
 
-40
 
 
 
-20
 
 
 
0
 
 
 
20
 
 
 
40
 
 
 
60
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [3]:
let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

//Sanitize data

//filter negative values
let mut negative_values= data_frame_polars.clone().lazy()
    .filter(col("age").lt_eq(0)).collect().unwrap();
// drop col    
// df.drop_in_place("age");
println!("{:?}",negative_values);
//Drop values if negative
let df= data_frame_polars.clone().lazy()
    .filter(col("age").gt_eq(0)).collect().unwrap();

println!("{:?}",df);

//mean of col

{
let mean_age = df.column("age").unwrap().f64().unwrap().mean().unwrap();
println!("{:?}",mean_age);
fn fill_negative_values(series: &Series,value:f64) -> Series {
    
    series.f64()
        .unwrap()
        .into_iter()
        .map(|v: Option<f64>| {
            let val = v.clone().unwrap();
            if val < 0.0 {
                
                return Some(value);
            }
            return v;
                
        })
       .collect::<Float64Chunked>()
    
    .into_series()

}

let fill_values = negative_values.apply("age",|s|fill_negative_values(s,mean_age));
println!("{:?}",fill_values);

}

shape: (3, 5)
┌──────────┬──────────────┬────────────┬─────────────┬─────────┐
│ clientid ┆ income       ┆ age        ┆ loan        ┆ default │
│ ---      ┆ ---          ┆ ---        ┆ ---         ┆ ---     │
│ i64      ┆ f64          ┆ f64        ┆ f64         ┆ i64     │
╞══════════╪══════════════╪════════════╪═════════════╪═════════╡
│ 16       ┆ 50501.726689 ┆ -28.218361 ┆ 3977.287432 ┆ 0       │
│ 22       ┆ 32197.620701 ┆ -52.42328  ┆ 4244.057136 ┆ 0       │
│ 27       ┆ 63287.038908 ┆ -36.496976 ┆ 9595.286289 ┆ 0       │
└──────────┴──────────────┴────────────┴─────────────┴─────────┘
shape: (1_994, 5)
┌──────────┬──────────────┬───────────┬─────────────┬─────────┐
│ clientid ┆ income       ┆ age       ┆ loan        ┆ default │
│ ---      ┆ ---          ┆ ---       ┆ ---         ┆ ---     │
│ i64      ┆ f64          ┆ f64       ┆ f64         ┆ i64     │
╞══════════╪══════════════╪═══════════╪═════════════╪═════════╡
│ 1        ┆ 66155.925095 ┆ 59.017015 ┆ 8106.532131 ┆ 0       │

()

In [134]:
//Previsores e Classes

let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();


let x_credit = data_frame_polars.clone().lazy()
    .select([col("*").exclude(["default","clientid"])])
    .with_columns([col("age").fill_null(lit(0))]).collect().unwrap();

let y_credit = data_frame_polars.clone().lazy()
    .select([col("default")]).collect().unwrap();

let ndarray_x = x_credit.clone().to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap();
let ndarray_y = y_credit.clone().to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();
let ndarray_y_flatten = Array::from_iter(ndarray_y.iter().cloned());
println!("{}",ndarray_x);
println!("{}",ndarray_y_flatten);
// {
// //get position in 2d array
// let get_index = ndarray_x.slice(s![.., 0]);
// let min_income_element = *get_index.iter().min_by(|a, b| a.total_cmp(b)).unwrap();
// let max_income_element = *get_index.iter().max_by(|a, b| a.total_cmp(b)).unwrap();

// let get_index = ndarray_x.slice(s![.., 1]);
// let min_age_element = *get_index.iter().min_by(|a, b| a.total_cmp(b)).unwrap();
// let max_age_element = *get_index.iter().max_by(|a, b| a.total_cmp(b)).unwrap();
    

// let get_index = ndarray_x.slice(s![.., 2]);
// let min_debit_element = *get_index.iter().min_by(|a, b| a.total_cmp(b)).unwrap();
// let max_debit_element = *get_index.iter().max_by(|a, b| a.total_cmp(b)).unwrap();

    
// println!("{} {} {}",min_income_element,min_age_element,min_debit_element);
// println!("{} {} {}",max_income_element,max_age_element,max_debit_element);

// // Standardisation -> Padrinizacao
// // x = x - avg(x) / detour(x)
// //
// // Normalization -> Normalizacao
// // x = x - min(x) / max(x) - min(x)


// // fn l1_norm_nd1(x: ArrayView1<f64>) -> f64 {
// //     x.fold(0., |acc, elem| acc + elem.abs())
// // }

    
// // fn l2_norm_nd1(x: ArrayView1<f64>) -> f64 {
// //     x.dot(&x).sqrt()
// // }

// // fn normalize_nd1(mut x: Array1<f64>) -> Array1<f64> {
// //     let norm = l2_norm_nd1(x.view());
// //     x.mapv_inplace(|e| e/norm);
// //     x
// // }

    
// }

{
//load dataset linda
let interpolate = Array1::from(vec![1.,1.,1.,1.]);
let dataset = Dataset::new(ndarray_x, interpolate);
    
// let scaler = NormScaler::l1();
// let dataset = scaler.transform(dataset);
let scaler = LinearScaler::standard().fit(&dataset).unwrap();
let dataset = scaler.transform(dataset);
    
println!("||x||_2 = {:?}", dataset.records);

    
}


[[66155.9250950813, 59.017015066929204, 8106.53213128514],
 [34415.1539658196, 48.11715310486029, 6564.745017677379],
 [57317.1700630337, 63.10804949188599, 8020.953296386469],
 [42709.534200839706, 45.751972352154596, 6103.642260140699],
 [66952.68884534019, 18.5843359269202, 8770.09923520439],
 ...,
 [59221.0448742844, 48.5181794138625, 1926.7293969037698],
 [69516.1275728606, 23.162104470655304, 3503.1761563262603],
 [44311.449262313494, 28.0171668957919, 5522.786693255141],
 [43756.0566049069, 63.971795841120205, 1622.72259832146],
 [69436.57955154781, 56.1526170284487, 7378.833598730591]]
[0, 0, 0, 0, 1, ..., 0, 0, 1, 0, 0]
||x||_2 = [[1.4539339314851303, 1.3334155763331506, 1.2028194204846896],
 [-0.7621755455355362, 0.5379303486644134, 0.6964269502960208],
 [0.8368207330749313, 1.6319842611717685, 1.1747114688630165],
 [-0.18307006098430073, 0.36531657097702286, 0.5449799947864358],
 [1.50956319426216, -1.6174106894374727, 1.4207648047234485],
 ...,
 [0.9697474031640506, 0.56719

()

In [219]:
let df = CsvReader::from_path("./db/census.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

println!("{}",df.clone());

// println!("{}",data_frame_polars.clone().null_count());

let df= df.clone().lazy();
let group= df.clone().filter(col("income").is_not_null()).collect()?.group_by(["income"])?.select(["income"]).count()?;
println!("{:?}",group);

let ndarray = group.clone().select(["income_count"])?.to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();
println!("{:?}",ndarray);

let minY= group.clone().column("income_count")?.min::<i32>().unwrap().unwrap();
let maxY= group.clone().column("income_count")?.max::<i32>().unwrap().unwrap();

let figure = evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count icome", ("sans-serif", 10.0))
        .build_cartesian_2d(0i32..4i32, minY..maxY)?;  
    
 chart.configure_mesh()
        .x_label_formatter(&|x| format!("{}", *x))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


    
 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(0)
            .data(ndarray
                    .indexed_iter()
                    .map(|((row, col), value)| (row as i32*3,*value))
                 )
                 )?;
    
    Ok(())
}).style("width:100%");

figure


shape: (32_562, 15)
┌──────┬──────────────────┬─────────┬────────────┬───┬──────┬────────────┬────────────────┬────────┐
│ age  ┆  employer        ┆  loan   ┆  education ┆ … ┆ non2 ┆  work_time ┆ country        ┆ income │
│ ---  ┆ ---              ┆ ---     ┆ ---        ┆   ┆ ---  ┆ ---        ┆ ---            ┆ ---    │
│ i64  ┆ str              ┆ str     ┆ str        ┆   ┆ str  ┆ str        ┆ str            ┆ str    │
╞══════╪══════════════════╪═════════╪════════════╪═══╪══════╪════════════╪════════════════╪════════╡
│ 39   ┆  State-gov       ┆  77516  ┆  Bachelors ┆ … ┆  0   ┆  40        ┆  United-States ┆  <=50K │
│ 50   ┆ Self-emp-not-inc ┆  83311  ┆  Bachelors ┆ … ┆  0   ┆  13        ┆  United-States ┆  <=50K │
│ 38   ┆  Private         ┆  215646 ┆  HS-grad   ┆ … ┆  0   ┆  40        ┆  United-States ┆  <=50K │
│ 53   ┆  Private         ┆  234721 ┆  11th      ┆ … ┆  0   ┆  40        ┆  United-States ┆  <=50K │
│ 28   ┆  Private         ┆  338409 ┆  Bachelors ┆ … ┆  0   ┆  40      

Count icome
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
8000
 
 
 
10000
 
 
 
12000
 
 
 
14000
 
 
 
16000
 
 
 
18000
 
 
 
20000
 
 
 
22000
 
 
 
24000
 
 
 
 
0
 
 
 
1
 
 
 
2
 
 
 
3
 
 
 
4

In [224]:
//age

let df = CsvReader::from_path("./db/census.csv").unwrap()
    .has_header(true)
    .finish().unwrap();


let df= df.clone().lazy();

let group= df.clone().filter(col("age").is_not_null()).collect()?.group_by(["age"])?.select(["age"]).count()?;
println!("{:?}",group);
let ndarray = group.clone().to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();

let minY= group.clone().column("age_count")?.min::<i32>().unwrap().unwrap();
let maxY= group.clone().column("age_count")?.max::<i32>().unwrap().unwrap();
let figure = evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count age", ("sans-serif", 10.0))
        .build_cartesian_2d(0i32..100i32, minY..maxY)?;  
    
 chart.configure_mesh()
        .x_label_formatter(&|x| format!("{}", *x))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


    
 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(0)
         .data(ndarray
                .rows()
                .into_iter()
                .map(|row|(row[0], row[1]))))?;
    
    Ok(())
}).style("width:100%");

figure


shape: (73, 2)
┌─────┬───────────┐
│ age ┆ age_count │
│ --- ┆ ---       │
│ i64 ┆ u32       │
╞═════╪═══════════╡
│ 24  ┆ 798       │
│ 21  ┆ 720       │
│ 30  ┆ 861       │
│ 18  ┆ 550       │
│ 27  ┆ 835       │
│ …   ┆ …         │
│ 80  ┆ 22        │
│ 71  ┆ 72        │
│ 68  ┆ 120       │
│ 74  ┆ 51        │
│ 83  ┆ 6         │
└─────┴───────────┘


Count age
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
100
 
 
 
200
 
 
 
300
 
 
 
400
 
 
 
500
 
 
 
600
 
 
 
700
 
 
 
800
 
 
 
 
0
 
 
 
10
 
 
 
20
 
 
 
30
 
 
 
40
 
 
 
50
 
 
 
60
 
 
 
70
 
 
 
80
 
 
 
90
 
 
 
100

In [46]:
//Label encoder, categorical Data

let df = CsvReader::from_path("./db/census.csv").unwrap()
    .has_header(true)
    .finish().unwrap();
println!("{:?}",df);

let df= df.clone().lazy();

let x_census = df.clone().select([
        col("*").exclude(["income"])
    // .cast(DataType::Categorical(None,CategoricalOrdering::Physical)),
        // col("education"),
    // .cast(DataType::Categorical(None,CategoricalOrdering::Physical)),
        // col("native-country")
    // .cast(DataType::Categorical(None,CategoricalOrdering::Physical))

    ])
    // .with_columns([
    //     col("workclass").to_physical(),
    //     col("education").to_physical(),
    //     col("native-country").to_physical()

    // ])
    .drop_nulls(None) .collect();


let y_census = df.clone().filter(col("income").is_not_null())
    .select([col("income").cast(DataType::Categorical(None,CategoricalOrdering::Physical))
            ])
    .with_columns([col("income").to_physical()])
    .drop_nulls(None)
    .collect();


let dummies_x = x_census?.clone().to_dummies(None, false).unwrap();

{
let ndarray_x = dummies_x.clone().to_ndarray::<Float32Type>(IndexOrder::Fortran).unwrap();
let ndarray_y = y_census?.clone().to_ndarray::<Float32Type>(IndexOrder::Fortran).unwrap();
let ndarray_y = Array::from_iter(ndarray_y.iter().cloned());

// let interpolate = Array1::from(vec![0.,1.,0.]);
// let dataset = Dataset::new(ndarray_x, interpolate);

// let scaler = LinearScaler::standard().fit(&dataset).unwrap();
// let dataset_x = scaler.transform(dataset);
// println!("{:?}",ndarray_x);


let n_samples = ndarray_x.shape()[0];
println!("samples {:?}",n_samples);
let test_size = 0.2; // 20% para teste
let test_size = (test_size * n_samples as f64).round() as usize;
println!("size {:?}",test_size);

let X_train = ndarray_x.slice(s![..n_samples - test_size, ..]).to_owned();
println!("X_train {:?}",X_train);

let X_test = ndarray_x.slice(s![n_samples - test_size.., ..]).to_owned();
println!("X_train {:?}",X_test);


let y_train = ndarray_y.slice(s![..n_samples - test_size]).to_owned();
println!("y_train {:?}",y_train);

let y_test = ndarray_y.slice(s![n_samples - test_size..]).to_owned();
println!("y_train {:?}",y_test);

    
    

}

shape: (32_562, 15)
┌──────┬─────────────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬────────┐
│ age  ┆ workclass   ┆ final-weig ┆ education  ┆ … ┆ capital-lo ┆ hour-per-w ┆ native-cou ┆ income │
│ ---  ┆ ---         ┆ ht         ┆ ---        ┆   ┆ ss         ┆ eek        ┆ ntry       ┆ ---    │
│ i64  ┆ str         ┆ ---        ┆ str        ┆   ┆ ---        ┆ ---        ┆ ---        ┆ str    │
│      ┆             ┆ str        ┆            ┆   ┆ str        ┆ str        ┆ str        ┆        │
╞══════╪═════════════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪════════╡
│ 39   ┆  State-gov  ┆  77516     ┆  Bachelors ┆ … ┆  0         ┆  40        ┆ United-Sta ┆  <=50K │
│      ┆             ┆            ┆            ┆   ┆            ┆            ┆ tes        ┆        │
│ 50   ┆ Self-emp-no ┆  83311     ┆  Bachelors ┆ … ┆  0         ┆  13        ┆ United-Sta ┆  <=50K │
│      ┆ t-inc       ┆            ┆            ┆   ┆            ┆      

()

In [36]:
// Same abordage using Linfa
let df = CsvReader::from_path("./db/census.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

let df= df.clone().lazy();



let x_census = df.clone().select([col("*").exclude(["income"])])
    .with_columns([
        col("workclass").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("final-weight").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("education").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("marital-status").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("occupation").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("relationship").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("race").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("sex").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("hour-per-week").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("capital-loss").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("capital-gain").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("education-num").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),


        col("native-country").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),        
    ])
    .drop_nulls(None).collect();
        println!("train {:?}",x_census);


let y_census = df.clone().filter(col("income").is_not_null())
    .select([col("income") ])
    .with_columns([
        col("income").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),      

    ])
    .drop_nulls(None)
    .collect();


// let dummies_x = x_census?.clone().to_dummies(None, false).unwrap();

{

let ndarray_x = x_census?.clone().to_ndarray::<Float32Type>(IndexOrder::C).unwrap();
let ndarray_y = y_census?.clone().to_ndarray::<Float32Type>(IndexOrder::C).unwrap();
let ndarray_y = Array::from_iter(ndarray_y.iter().cloned());

    let ds = Dataset::new(ndarray_x, ndarray_y);
    {
    let (train, test) = ds.split_with_ratio(0.7);
    println!("train {:?}",train);
    println!("test {:?}",test);
    }
}

train Ok(shape: (32_561, 14)
┌─────┬───────────┬────────────┬───────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ age ┆ workclass ┆ final-weig ┆ education ┆ … ┆ capital-ga ┆ capital-lo ┆ hour-per-w ┆ native-cou │
│ --- ┆ ---       ┆ ht         ┆ ---       ┆   ┆ in         ┆ ss         ┆ eek        ┆ ntry       │
│ i64 ┆ u32       ┆ ---        ┆ u32       ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│     ┆           ┆ u32        ┆           ┆   ┆ u32        ┆ u32        ┆ u32        ┆ u32        │
╞═════╪═══════════╪════════════╪═══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 39  ┆ 0         ┆ 0          ┆ 0         ┆ … ┆ 0          ┆ 0          ┆ 0          ┆ 0          │
│ 50  ┆ 1         ┆ 1          ┆ 0         ┆ … ┆ 1          ┆ 0          ┆ 1          ┆ 0          │
│ 38  ┆ 2         ┆ 2          ┆ 1         ┆ … ┆ 1          ┆ 0          ┆ 0          ┆ 0          │
│ 53  ┆ 2         ┆ 3          ┆ 2         ┆ … ┆ 1          ┆ 

()